# Лемматизация с udpipe

## 1. Считаем файл

In [ ]:
filename = 'grc.txt' #файл без диакритики
#filename = 'csl.txt' #файл с диакритикой

## выгрузим текст из файла
with open(filename) as open_file:
    text_as_lines = open_file.readlines()

In [ ]:
import requests

## 2. Подгрузим модели

In [ ]:
response = requests.get("http://lindat.mff.cuni.cz/services/udpipe/api/models")
info = response.json()
info

In [ ]:
for key, data in info["models"].items():
    #if "russian" in key:
    if "greek" in key:
        print(key, data)

#моедли для русского:
#old_russian-torot-ud-2.6-200830 ['tokenizer', 'tagger', 'parser']
#old_russian-rnc-ud-2.6-200830 ['tokenizer', 'tagger', 'parser'] - довольно низкое качество?

#следующие три модели по качеству очень схожи, но кажется taiga чуть лучше 
#russian-syntagrus-ud-2.6-200830 ['tokenizer', 'tagger', 'parser']
#russian-gsd-ud-2.6-200830 ['tokenizer', 'tagger', 'parser']
#russian-taiga-ud-2.6-200830 ['tokenizer', 'tagger', 'parser']


#модели для греческого:
#ancient_greek-perseus-ud-2.6-200830 ['tokenizer', 'tagger', 'parser']
#ancient_greek-proiel-ud-2.6-200830 ['tokenizer', 'tagger', 'parser'] 
#greek-gdt-ud-2.6-200830 ['tokenizer', 'tagger', 'parser']

In [ ]:
params = {"tokenizer": "", "tagger": "", "parser": "", "model": "greek-gdt-ud-2.6-200830"}

## 3. Применяем модель к тексту и записываем результаты работы парсера в файл

In [ ]:
i = 0
for line in tqdm(text_as_lines):
    params["data"] = line
    response = requests.get("http://lindat.mff.cuni.cz/services/udpipe/api/process", params)
    json_response = response.json()
    parse = json_response["result"]
    if i == 0:
        with open('greek_gtd_ud.txt', 'w') as my_file:
            my_file.write(parse)
    else:
        with open('greek_gtd_ud.txt', 'a') as my_file:
            my_file.write(parse)
    i += 1

## 4. Переходим к работе с этим файлом

In [ ]:
file_to_scretch = 'greek_gtd_ud.txt'

In [ ]:
with open(file_to_scretch) as open_file:
    text_as_lines = open_file.readlines()

In [ ]:
text_as_lines[:10]

#### в некоторых моделях знаки препинания не отделяются от слова - хотя чаще в ud все же они содержат собственную метаинформацию (пока что такой была только torot для csl - в таком случае итоговый файл с леммами сохранится в формате string, но его можно легко почистить)

In [ ]:
## эту ячейку в будущем нужно заменить на регулярки
numbers = list()
i = 0
for i in range(0, 100):
    i = str(i)
    numbers.append(i)
    i = int(i)
    i += 1

### Проверяем, что строка начинается с цифры (результат работы парсера) и если это так, добавляем ее в новый список. Этот список и будущий файл с ним содержит всю распарсенную информацию. На следующем этапе мы достанем только леммы

In [ ]:
list_lemmas_data = []

for line in text_as_lines:
    if line[0] in numbers:
        list_lemmas_data.append(line)

In [ ]:
list_lemmas_data_split = []

for line in list_lemmas_data:
    new_line = line.split('\t')
    list_lemmas_data_split.append(new_line)

## Теперь достанем леммы - это третий (2) элемент каждого списка list_lemmas_data_split

In [ ]:
list_lemmas_data_split[:15]

In [ ]:
lemma_list_or_rnc = []

for token_list in list_lemmas_data_split:
    lemma = token_list[2]
    lemma_list_or_rnc.append(lemma)

### Запишем результаты в файл

In [ ]:
i = 0 
for item in lemma_list_or_rnc:
    if i == 0:
        with open('lemmas_greek_gtd_ud.txt', 'w') as file:
            if item != '.':
                file.write(item + ' ')
    else:
        with open('lemmas_greek_gtd_ud.txt', 'a') as file:
            if item != '.':
                file.write(item + ' ')
            else:
                file.write('\n')
    i += 1